# SteamDB scraping

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup
import pandas as pd
import requests
import json

## Open site

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\gianl\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\gianl\AppData\Local\Temp/ipykernel_18096/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
driver.get('https://steamdb.info/graph/')

In [4]:
select = Select(WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.NAME, "table-apps_length"))))
select.select_by_value('-1') #show "all" results

## Scraping

### Obtain list Id Games

In [5]:
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

In [6]:
games = soup.find_all("tr", class_='app')

In [7]:
id_games = [int(game['data-appid']) for game in games]
len(id_games)

5648

### Requests all games

Create dictionary with relevant data for every game

In [38]:
steam_games = {}

for count, i in enumerate(id_games):
    print(count+1, end= "\r")
    
    
    driver.get(f'https://steamdb.info/api/RenderAppHover/?appid={i}')
    html = driver.page_source
    window = BeautifulSoup(html, "lxml")
    
    if 'HTTP ERROR 406' in html:
        pass
    else:
        steam_games[i] = {}
        
        #title of game
        steam_games[i]['title'] = window.find('a',class_='hover_title').text

        #info game
        for elem in window.find_all('div', class_='hover_body hover_meta'):
            if elem.find('b') is not None:
                steam_games[i][elem.text.split(':')[0].strip('\n')] = elem.find('b').text

        #categories
        if window.find('div',class_='hover_body hover_tag_row') is not None:
            steam_games[i]['categories'] = window.find('div',class_='hover_body hover_tag_row').text.strip().split('\n')

In [39]:
print('Total games: \t', len(id_games))
print('Scraped games: \t', len(steam_games))

Total games: 	 5599
Scraped games: 	 3198


In [46]:
print(json.dumps(steam_games[i], indent = 4, default= str))

{
    "title": "Leif's Adventure: Netherworld Hero Demo",
    "Developer": "OneManOnMars Art &amp; Games e.K.",
    "Publisher": "OneManOnMars Art & Games e.K.",
    "Release Date": "Unknown",
    "Last Update": "13 June 2022",
    "24h Player Peak": "1",
    "Followers": "0"
}


## Export

In [41]:
with open('steam_games.json', 'w') as writefile:
    writefile.write(json.dumps(steam_games))